<a href="https://colab.research.google.com/github/igorgatchin1993/assigments/blob/main/Gatchin_Igor_assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Theory

In the following assignment, your task is to complete the MNIST Basics chapter. It is best to repeat everything from last week and try to answer the following questions. Afterwards you have to summarize the learned facts with two programming tasks.

What is "torch.cat()" and ".view(-1, 28*28)" doing in the beginning of the "The MNIST Loss Function" chapter?

In the beginning of "The MNIST Loss Function" chapter, the torch.cat() function is used to concatenate the batch of images into a single tensor.






What is the defined `mnist_loss` function doing? 


```
def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()
```

In [ ]:
The mnist_loss function is a loss function for the task of image classification using the MNIST dataset, which contains grayscale images of hand-written digits (0 to 9). The function takes two arguments:

predictions: A tensor of shape (batch_size, num_classes) containing the predicted probabilities for each class for each image in the batch.
targets: A tensor of shape (batch_size, num_classes) containing the one-hot encoded target labels for each image in the batch.
The function first uses the torch.where function to create a new tensor of the same shape as predictions. For each element of targets that is equal to 1, the corresponding element in the new tensor is set to 1-predictions, and for each element of targets that is equal to 0, the corresponding element in the new tensor is set to predictions.

The result of the torch.where function is then averaged over all elements of the tensor using the mean method. This gives the mean cross-entropy loss between the predicted probabilities and the target labels for the batch.

Can you draw the neuronal network, which is manually trained in chapter "The MNIST Loss Function"?


Here is an example code for creating a simple fully connected neural network with one hidden layer and using the mnist_loss function for training:

In [ ]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Define the neural network architecture
class NeuralNet(torch.nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = torch.nn.Linear(784, 128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define the loss function
def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()

# Load the MNIST dataset
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Initialize the neural network and optimizer
model = NeuralNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Train the neural network
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Flatten the images into a batch of vectors
        images = images.view(-1, 28*28)

        # Forward pass
        predictions = model(images)

        # Compute the loss
        targets = torch.zeros(len(labels), 10)
        targets[range(len(labels)), labels] = 1
        loss = mnist_loss(predictions, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the trained model on the test set
with torch.no_grad():
    total_correct = 0
    total_images = 0
    for images, labels in test_loader:
        # Flatten the images into a batch of vectors
        images = images.view(-1, 28*28)

        # Forward pass and compute the predictions
        predictions = model(images)
        _, predicted = torch.max(predictions, 1)

        # Compute the accuracy on the test set
        total_correct += (predicted == labels).sum().item()
        total_images += labels.size(0)

    accuracy = total_correct / total_images
    print(f'Test Accuracy: {accuracy:.4f}')


Why is it not possible to use the accuracy as loss function?

While accuracy is an important metric for evaluating the performance of a machine learning model, it cannot be used as a loss function directly for optimization during the training process. There are several reasons why:

1. Non-differentiability: Accuracy is a measure of how well the model predicts the correct class labels, but it is not a differentiable function. This means that small changes in the model parameters cannot be used to calculate a gradient that can be used to update the parameters during training using gradient descent or other optimization algorithms.

2. Discrete nature of accuracy: Accuracy is a discrete measure, meaning that it can only take on certain values, such as 0%, 10%, 20%, etc. This makes it difficult to use as a continuous loss function that can be optimized through gradient descent.

3. Imbalance of classes: In many classification tasks, the classes may be imbalanced, meaning that there are more examples of one class than the other. In such cases, a model that always predicts the majority class will have a high accuracy but will not be useful. Therefore, other metrics such as precision, recall, F1-score or AUC-ROC, are used to take into account the imbalance of classes and provide a more complete evaluation of the model performance.

For these reasons, machine learning models typically use other loss functions such as cross-entropy, mean squared error, or hinge loss, which are differentiable and can be optimized during training to improve the model's performance.

What is the defined `mnist_loss` function doing? 

One commonly used loss function for this task is the cross-entropy loss function, which measures the dissimilarity between the predicted class probabilities and the true class labels. Another possible loss function is the mean squared error loss, which measures the average squared difference between the predicted outputs and the true outputs.

Why do we need additionaly the sigmoid() function? What is it technically in our TLU?

Sigmoid Function in TLU:
The sigmoid function is a mathematical function that maps any input value to a value between 0 and 1. In the context of a Threshold Logic Unit (TLU), the sigmoid function is used as an activation function that introduces nonlinearity into the output of the TLU. The TLU performs a weighted sum of its inputs, and the sigmoid function is applied to this sum to produce the output. The sigmoid function can be expressed mathematically as:

sigmoid(x) = 1 / (1 + exp(-x))

where x is the input to the function. The output of the TLU with the sigmoid activation function can be interpreted as a probability of the output being 1 or 0, depending on the threshold set.



Again, what are mini batches, why are we using them and why should they be shuffeld?

Mini-Batches:
Mini-batch training is a technique used in machine learning to train models using small subsets of the training data at a time. Instead of using the entire training dataset for each iteration of the optimization algorithm, mini-batch training divides the dataset into smaller batches of data, which are then used to update the model parameters. These batches are typically chosen randomly from the dataset, and each batch contains a fixed number of examples. By using mini-batches, the model can be trained more efficiently, as it can update its parameters more frequently, and the stochastic nature of the batch selection can help the model avoid getting stuck in local minima.

Shuffling Mini-Batches:
Shuffling the mini-batches is important to avoid any systematic patterns that may exist in the data. For example, if the data is sorted by class labels, then using batches that follow this order will lead to a biased model. Shuffling the batches ensures that the model is exposed to a more representative sample of the data in each iteration of the optimization algorithm, which can help to improve the generalization performance of the model. Additionally, shuffling helps to break up any dependencies or correlations that may exist between the examples in the dataset, which can also help to improve the learning process.

#Practical Part

Try to understand all parts of the code needed to manually train a single TLU/Perceptron, so use and copy all parts of the code from "First Try: Pixel Similarity" to the "Putting it all together" chapter. In the second step, use an optimizer, a second layer, and a ReLU as a hidden activation function to train a simple neural network. When copying the code, think carefully about what you really need and how you can summarize it as compactly as possible. (Probably each attempt requires about 15 lines of code.)

In [2]:
#YOUR TASK: Manually train a single layer perceptron without using an optimizer.
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
mnist = fetch_openml('mnist_784')
X, y = mnist['data'], mnist['target']

# Preprocess the data
X = X.astype(np.float32) / 255.0
y = y.astype(np.int32)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
n_inputs = X_train.shape[1]
n_outputs = 10  # number of classes
weights = np.random.randn(n_inputs, n_outputs)
bias = np.zeros(n_outputs)

# Define the hyperparameters
learning_rate = 0.1
n_epochs = 50
batch_size = 32

# Train the model
for epoch in range(n_epochs):
    # Shuffle the training data
    idx = np.random.permutation(len(X_train))
    X_train_shuffled = X_train[idx]
    y_train_shuffled = y_train[idx]
    
    # Loop over mini-batches
    for i in range(0, len(X_train_shuffled), batch_size):
        X_batch = X_train_shuffled[i:i+batch_size]
        y_batch = y_train_shuffled[i:i+batch_size]
        
        # Compute the forward pass
        z = np.dot(X_batch, weights) + bias
        y_pred = np.argmax(z, axis=1)
        
        # Compute the loss
        loss = np.mean(y_batch != y_pred)
        
        # Compute the backward pass
        grad_z = (y_pred - y_batch).reshape(-1, 1)
        grad_weights = np.dot(X_batch.T, grad_z)
        grad_bias = np.sum(grad_z, axis=0)
        
        # Update the weights and bias
        weights -= learning_rate * grad_weights
        bias -= learning_rate * grad_bias
        
    # Compute the validation accuracy
    z_val = np.dot(X_val, weights) + bias
    y_val_pred = np.argmax(z_val, axis=1)
    val_acc = np.mean(y_val == y_val_pred)
    
    print('Epoch: {}, Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch, loss, val_acc))


/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


KeyboardInterrupt: ignored

This code defines a single-layer perceptron with weights and bias initialized randomly, and trains it on the MNIST dataset using mini-batch stochastic gradient descent. The code shuffles the training data at the beginning of each epoch to ensure the model sees a representative sample of the data in each iteration. The loss is computed as the mean classification error between the predicted and true labels. The weights and bias are updated using the gradients computed by the backward pass. The validation accuracy is computed at the end of each epoch to monitor the generalization performance of the model.

In [ ]:
#YOUR TASK: Train a simple two-layer neural network (two perceptrons + hidden activation function) with built-in functions and an optimizer.

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms

# Define the network architecture
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64) # input layer to hidden layer
        self.fc2 = nn.Linear(64, 10) # hidden layer to output layer
        self.relu = nn.ReLU() # activation function
        
    def forward(self, x):
        x = x.view(x.shape[0], -1) # flatten the input tensor
        x = self.relu(self.fc1(x)) # pass through hidden layer with ReLU activation function
        x = self.fc2(x) # pass through output layer
        return x

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the network
net = Net()
for epoch in range(5): # number of epochs
    running_loss = 0
    for images, labels in trainloader: # loop through mini-batches
        optimizer.zero_grad() # zero the gradients
        output = net(images) # forward pass
        loss = criterion(output, labels) # calculate loss
        loss.backward() # backward pass
        optimizer.step() # update weights
        running_loss += loss.item() # calculate running loss
        
    print(f"Epoch {epoch+1} - Training loss: {running_loss/len(trainloader)}")


This code defines a Net class for the network architecture, which consists of two fully connected layers with a ReLU activation function in between. The code then loads the MNIST dataset and defines the loss function and optimizer. Finally, the network is trained using a for loop over the number of epochs and a nested for loop over mini-batches. The optimizer is used to update the weights of the network during training.